In [1]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta
import seaborn as sns
import boto3
import matplotlib.pyplot as plt

In [23]:
import sys

In [2]:
s3 = boto3.client('s3')

In [3]:
file = s3.get_object(Bucket='sf-ems-analysis', Key='woring_3h_lim.csv')

In [4]:
df = pd.read_csv(file['Body'])

In [5]:
df.set_index(keys=df['Received DtTm'], inplace = True)

In [6]:
df.drop(columns=['Received DtTm'], inplace = True)

In [7]:
from scipy import signal
from scipy import stats
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [24]:
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
 
# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]
 
# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))
 
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	models = list()
	# define config lists
	p_params = [0, 1]
	d_params = [0, 1]
	q_params = [0, 1]
	t_params = ['t']
	P_params = [1,2]
	D_params = [0, 1]
	Q_params = [0, 1]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models



In [ ]:
data = df[25192-(56*4):25192+56].values
print(data)
# data split
n_test = 56
# model configs
cfg_list = sarima_configs(seasonal=[8])
# grid search
scores = grid_search(data, cfg_list, n_test)
print('done')
# list top 3 configs
for cfg, error in scores[:3]:
    print(cfg, error)

[[33]
 [26]
 [31]
 [62]
 [67]
 [67]
 [66]
 [53]
 [24]
 [26]
 [48]
 [57]
 [67]
 [72]
 [57]
 [49]
 [40]
 [26]
 [41]
 [53]
 [68]
 [69]
 [61]
 [51]
 [45]
 [32]
 [34]
 [53]
 [74]
 [55]
 [72]
 [51]
 [24]
 [17]
 [33]
 [47]
 [69]
 [73]
 [65]
 [46]
 [28]
 [18]
 [39]
 [56]
 [68]
 [64]
 [57]
 [51]
 [40]
 [39]
 [41]
 [51]
 [77]
 [75]
 [62]
 [44]
 [36]
 [20]
 [48]
 [48]
 [73]
 [76]
 [63]
 [62]
 [32]
 [24]
 [50]
 [53]
 [55]
 [74]
 [55]
 [43]
 [47]
 [28]
 [33]
 [54]
 [58]
 [54]
 [52]
 [61]
 [47]
 [24]
 [39]
 [47]
 [46]
 [49]
 [61]
 [52]
 [27]
 [22]
 [38]
 [70]
 [69]
 [62]
 [46]
 [24]
 [28]
 [22]
 [46]
 [59]
 [57]
 [51]
 [60]
 [41]
 [26]
 [27]
 [32]
 [61]
 [55]
 [65]
 [58]
 [41]
 [27]
 [26]
 [37]
 [61]
 [61]
 [62]
 [77]
 [46]
 [36]
 [24]
 [40]
 [71]
 [64]
 [74]
 [72]
 [51]
 [48]
 [22]
 [30]
 [67]
 [52]
 [70]
 [67]
 [53]
 [45]
 [28]
 [33]
 [49]
 [57]
 [67]
 [52]
 [46]
 [39]
 [20]
 [47]
 [52]
 [63]
 [73]
 [68]
 [53]
 [34]
 [34]
 [45]
 [66]
 [56]
 [69]
 [56]
 [61]
 [32]
 [18]
 [35]
 [66]
 [68]
 [66]
 [58